In [1]:
import numpy as np
import h5py as h5
import matplotlib.pyplot as plt
import sys
sys.path.append(r"C:\Users\hyli93\Documents\GitHub\CrystalDiffraction")

from CrystalDiff import util, crystal, pulse, auxiliary

In [2]:
# First construct the original k mesh
energy_center = 10.
k_len = util.kev_to_wave_number(energy=energy_center)

# Get the momentum mesh
number_x = 500
number_y = 500
number_z = 10 ** 5
(kx_grid,
 ky_grid, 
 kz_grid, 
 axis_info) = auxiliary.get_k_mesh_3d(number_x=number_x,
                                      number_y=number_y,
                                      number_z=number_z,
                                      delta_e_x=1e-3,
                                      delta_e_y=1e-3,
                                      delta_e_z=1e-3/util.c)

# 2nd. Use the fact that we have downsampled the real space by a factor of 2.
x_ds = axis_info['x_idx'][::2]
y_ds = axis_info['y_idx'][::2]
z_ds = axis_info['z_idx'][number_z // 2: number_z//2 + 500:2]

# 3rd. Construct the corresponding momentum mesh from the position mesh
kx_range = np.pi * 2 / (x_ds[1] - x_ds[0])
kx_ds = np.linspace( - kx_range/2, kx_range/2, 250)

ky_range = np.pi * 2 / (y_ds[1] - y_ds[0])
ky_ds = np.linspace( - ky_range/2, ky_range/2, 250)

kz_range = np.pi * 2 / (z_ds[1] - z_ds[0])
kz_ds = np.linspace( - kz_range/2, kz_range/2, 250)

kz_ds += k_len

# Load data

In [10]:
with h5.File("C:/Users/hyli93/Downloads/asymmetric24_part_3_y_field_ds.h5") as h5file:
    data = np.array(h5file['field'])

# Get Energy variance and the average momentum

In [11]:
spectrum = np.fft.fftshift(np.fft.fftn(np.fft.ifftshift(data)))
spectrum = np.square(np.abs(spectrum))

# Convert the intensity to probability
momentum_distribution = spectrum / np.sum(spectrum) 

In [16]:
# Get average k
momentum_distribution_x = np.sum(momentum_distribution, axis=(1,2))
momentum_distribution_y = np.sum(momentum_distribution, axis=(0,2))
momentum_distribution_z = np.sum(momentum_distribution, axis=(0,1))

mean_kx = np.sum(np.multiply(momentum_distribution_x, kx_ds))
mean_ky = np.sum(np.multiply(momentum_distribution_y, ky_ds))
mean_kz = np.sum(np.multiply(momentum_distribution_z, kz_ds))

mean_kx_square = np.sum(np.multiply(momentum_distribution_x, np.square(kx_ds)))
mean_ky_square = np.sum(np.multiply(momentum_distribution_y, np.square(ky_ds)))
mean_kz_square = np.sum(np.multiply(momentum_distribution_z, np.square(kz_ds)))

mean_k = np.sqrt(mean_kx_square + mean_ky_square + mean_kz_square)

var_kx = mean_kx_square - mean_kx ** 2
var_ky = mean_ky_square - mean_ky ** 2
var_kz = mean_kz_square - mean_kz ** 2

var_k = var_kx + var_ky + var_kz

delta_energy = util.petahertz_angular_frequency_to_kev(np.sqrt(var_k))
print("The energy variance is {:.4e} eV".format(delta_energy * 1e3))

The energy variance is 5.8822e-01 eV


In [15]:
delta_theta_x = 2 * np.arcsin(np.sqrt(var_kx) / mean_k)
delta_theta_y = 2 * np.arcsin(np.sqrt(var_ky) / mean_k)

print("Delta theta x is {:.4e} rad".format(delta_theta_x))
print("Delta theta y is {:.4e} rad".format(delta_theta_y))

Delta theta x is 9.3267e-07 rad
Delta theta y is 9.3230e-07 rad


# Get spectrum

In [18]:
# Get the 2D probability density distribution
momentum_distribution_xy = np.sum(momentum_distribution, axis=-1)

sigma_mat, eig, eig_vec = util.get_2d_sigma_matrix(density_2d=momentum_distribution_xy,
                                                    x_values= kx_ds, 
                                                    y_values= ky_ds)

print(sigma_mat)
print(eig)
print(eig_vec)

[[  5.58499349e-04  -7.54875763e-18]
 [ -7.54875763e-18   5.58054180e-04]]
[ 0.0005585   0.00055805]
[[  1.00000000e+00   1.69570671e-11]
 [ -1.69570671e-11   1.00000000e+00]]


# Get position space statistics

In [19]:
# Convert the magnitude to the intensity
distribution = np.square(np.abs(data))

# Convert the intensity to probability
distribution /= np.sum(distribution) 

In [20]:
# Get the 2D probability density distribution
distribution_2d = np.sum(distribution, axis=-1)

sigma_mat, eig, eig_vec = util.get_2d_sigma_matrix(density_2d=distribution_2d,
                                                    x_values= x_ds, 
                                                    y_values= y_ds)

In [21]:
print(sigma_mat)
print(eig)
print(eig_vec)

[[  4.51230722e+02   2.69631097e-11]
 [  2.69631097e-11   4.63304833e+02]]
[ 451.2307218   463.30483285]
[[ -1.00000000e+00  -2.23313415e-12]
 [  2.23313415e-12  -1.00000000e+00]]
